# **Feture Construction and Splitting**

# Feature Construction

In [20]:
import numpy as np
import pandas as pd


from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns


In [21]:
df=pd.read_csv(r"A:\Work Docs\Data Analyst work\Campus X\00 Datasets For coding\titanic_train.csv")[['Age','Pclass','SibSp','Parch','Survived']]

In [22]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [23]:
#Removing Missing Data
df.dropna(inplace=True)

In [24]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [25]:
x=df.iloc[:,0:4]
y=df.iloc[:,-1]

In [26]:
x.head()

,Age,Pclass,SibSp,Parch
0,22.0,3,1,0
1,38.0,1,1,0
2,26.0,3,0,0
3,35.0,1,1,0
4,35.0,3,0,0


In [27]:
np.mean(cross_val_score(LogisticRegression(),x,y,scoring='accuracy',cv=20))

0.6933333333333332

# Applying Feature Construction
### Family = Sibling/Spouse + Parent/Child + 1 (Passenger HimSelf)

In [28]:
x['Family_size']=x['SibSp']+x['Parch']+1

In [29]:
x.head()

,Age,Pclass,SibSp,Parch,Family_size
0,22.0,3,1,0,2
1,38.0,1,1,0,2
2,26.0,3,0,0,1
3,35.0,1,1,0,2
4,35.0,3,0,0,1


### Creating Function: If person is Alone (family=1) it will return 0, if Family is >1 and<=4, it will return 1, Family>4 it will return 2

In [30]:
def myfunc(num):
    if num==1:
        return 0 #Alone
    elif num > 1 and num <= 4:
        return 1 #small Family
    else:
        return 2 #Large Family

In [31]:
myfunc(4)

1

In [32]:
x['Family_type']=x['Family_size'].apply(myfunc)

In [33]:
x.sample(5)

,Age,Pclass,SibSp,Parch,Family_size,Family_type
534,30.0,3,0,0,1,0
345,24.0,2,0,0,1,0
514,24.0,3,0,0,1,0
821,27.0,3,0,0,1,0
638,41.0,3,0,5,6,2


In [34]:
x.drop(columns=['SibSp','Parch','Family_size'],inplace=True)

In [35]:
x.sample(5)

,Age,Pclass,Family_type
299,50.0,1,1
412,33.0,1,1
730,29.0,1,0
523,44.0,1,1
508,28.0,3,0


In [36]:
np.mean(cross_val_score(LogisticRegression(),x,y,scoring='accuracy',cv=20))

0.7003174603174602

# Feature Splitting

In [37]:

df=pd.read_csv(r"A:\Work Docs\Data Analyst work\Campus X\00 Datasets For coding\titanic_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
df["Title"]=df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

In [39]:
df[['Title','Name']]

,Title,Name
0,Mr,"Braund, Mr. Owen Harris"
1,Mrs,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,Miss,"Heikkinen, Miss. Laina"
3,Mrs,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,Mr,"Allen, Mr. William Henry"
...,...,...
886,Rev,"Montvila, Rev. Juozas"
887,Miss,"Graham, Miss. Margaret Edith"
888,Miss,"Johnston, Miss. Catherine Helen ""Carrie"""
889,Mr,"Behr, Mr. Karl Howell"


In [40]:
df.groupby('Title')

In [41]:
(df.groupby('Title')['Survived'].mean()*100).reset_index().sort_values(by='Survived',ascending=False)

,Title,Survived
16,the Countess,100.000000
9,Mlle,100.000000
15,Sir,100.000000
13,Ms,100.000000
5,Lady,100.000000
10,Mme,100.000000
12,Mrs,79.200000
8,Miss,69.780220
7,Master,57.500000
1,Col,50.000000


In [42]:
df['Is_Married']=0
df['Is_Married'].loc[df['Title']== 'Mrs']=1
df['Is_Married']

C:\Users\acz\AppData\Local\Temp\ipykernel_19040\195764523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_Married'].loc[df['Title']== 'Mrs']=1


0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    0
888    0
889    0
890    0
Name: Is_Married, Length: 891, dtype: int64